In [ ]:
import pyfmi
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
##PATH TO FMU FILE: 
pth = "../fmus/"
fnm = "hex_delta.fmu"
print(pth+fnm)

In [ ]:
#CREATE AN FMU model OBJECT:
fmu = pyfmi.load_fmu(pth+fnm)
fmu

In [ ]:
## LIST THE VARIABLES ASSOCIATED WITH THE MODEL:
model_variables = fmu.get_model_variables()
print(model_variables)
variable_names = fmu.get_model_variables().keys()

In [ ]:
## PRINT THE DEFAULT VALUES OF THE VARIABLES STORED IN THE FMU FILE:
def print_variables(fmu):
    variable_names = fmu.get_model_variables().keys()
    for variable_name in variable_names:
        value = fmu.get(variable_name)
        print(variable_name + ': '+ str(value))


In [ ]:
## SET THE INITIAL CONDITIONS FOR THE OUTPUT TEMPERATURES:
T1_ini = 91
T2_ini = 68

fmu.set('T1_ini',T1_ini)
fmu.set('T2_ini',T2_ini)

print_variables(fmu)

In [ ]:
## SETTING THE SIMULATION TIMES:
final_time = 60.0 
step_size = .1
nsteps = int(final_time/step_size) #number of time steps
times = np.arange(0,final_time,step_size)

# SIMULATION OPTIONS
opts = fmu.simulate_options()
opts["ncp"] = nsteps # number of output points

In [ ]:
## DEFINE INPUT OBJECT for SIMULATION
T1_ins = T1_ini*np.ones(nsteps)
T2_ins = T2_ini*np.ones(nsteps)
m1_ins = 56*np.ones(nsteps)
m2_ins = 46*np.ones(nsteps)
u = np.transpose(np.vstack((times,T1_ins,T2_ins,m1_ins,m2_ins)))
# Simulate the model response , given the initial parameters
input_labels = ['T1_in','T2_in','m1_in','m2_in']
input_object = (input_labels,u)

In [ ]:
#SIMULATE:
res = fmu.simulate(final_time=final_time,options=opts, input=input_object)

In [ ]:
#PLOT THE RESULTS:
T1_outs = np.array(res['T1_out'])
T2_outs = np.array(res['T2_out'])

fig,ax = plt.subplots()
ax.plot(res['time'],T1_outs,'*-',label='T1_out')
ax.plot(res['time'],T2_outs,'*-',label='T2_out')
ax.grid()
plt.xlabel('time(sec)')
plt.ylabel('Temperature (°C)')
plt.title('Output Temperatures')
ax.legend()